In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os
import random
import numpy as np
import sys
from multiprocessing import Pool
import gc
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import random
import gzip

In [2]:
spark = SparkSession.builder.appName('Dataframe').getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.conf.set('spark.sql.debug.maxToStringFields',100000)

22/05/26 12:03:54 WARN Utils: Your hostname, dsaserver resolves to a loopback address: 127.0.0.1; using 172.16.1.40 instead (on interface eth0)
22/05/26 12:03:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 12:03:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
files= os.listdir('/s3data')
lst = ['/s3data/'+i for i in files]
del files
gc.collect()
samples = random.sample(lst,35700)
del lst
gc.collect()

0

In [4]:
%%time
df = spark.read.json(samples)

CPU times: user 3.1 s, sys: 1.41 s, total: 4.51 s
Wall time: 11min 47s


In [ ]:
df.count()